# Visión por Computador - Práctica 3
## Cámara, geometría epipolar y reconstrucción estérea

### Estimación de la matriz de una cámara a partir del conjunto de puntos en correspondencias.

##### a) Generar la matriz de una cámara finita P a partir de valores aleatorios en [0,1] . Verificar si representa una cámara finita y en ese caso quedársela.
##### b) Suponer un patrón de puntos del mundo 3D compuesto por el conjunto de puntos con coordenadas $\{(0,x_1,x_2)$ y $(x_2,x_1,0)$, para $x_1=0.1:0.1:1$ y $x_2=0.1:0.1:1\}$. Esto supone una rejilla de puntos en dos planos distintos ortogonales.
##### c) Proyectar el conjunto de puntos del mundo con la cámara simulada y obtener las coordenadas píxel de su proyección.
##### d) Implementar el algoritmo DLT para estimar la cámara P a partir de los puntos 3D y sus proyecciones en la imagen.
##### e) Calcular el error de la estimación usando la norma de Frobenius (cuadrática).
##### f) Mostrar en una única imagen los puntos 3D proyectados con la cámara estimada y la cámara simulada.

*  **Apartado a:** para generar una cámara finita $P = K[R|T]$, donde $P=[KR|KT] = [M|M_T]$ simplemente tenemos que generar una matriz con números aleatorios y asegurarnos de que el determinante de $M$ sea distinto de 0. Al final, devolveremos una matriz de tamaño $3\times4$ con $det(M)\neq0$.

In [1]:
import cv2
import math
import numpy as np
import functions as fx

def generate_Pcamera():
    # La matriz cámara tiene la siguiente estructura:
    #              P =  K[R | T]
    # donde det(R) != 0, al igual que det(K) != 0, por
    # lo que podemos hacer que P = [KR | KT] = [M | M_4]

    # Generamos una matriz con valores aleatorios en el
    # intervalo [0,1)
    P_cam = np.random.rand(3,4)

    # Comprobamos si det(M) != 0. En caso de que no lo
    # sea, volvemos a generar una nueva matriz cámara.
    while not np.linalg.det(P_cam[0:3,0:3]):
        P_cam = np.random.rand(3,4)

    P_cam = P_cam / P_cam[-1,-1]
    return P_cam

* **Apartado b:** para generar los puntos con coordenadas $\{(0,x_1,x_2)$ y $(x_2,x_1,0)$, para $x_1=0.1:0.1:1$ y $x_2=0.1:0.1:1\}$, simplemente generamos dos arrays de 0 a 1 en intervalos de 0.1, que concatenaremos en forma de columna y añadiremos un cero por la izquierda o la derecha, según corresponda, generando puntos de la siguiente forma:
$$set_1 = \left[\begin{matrix}
  0 & 0 & 0 \\
  0 & 0 & 0.1 \\
   & \vdots &  \\
  0 & 0.1 & 0.5 \\
  & \vdots & \\
  0 & 0.9 & 0.9
 \end{matrix}\right] \qquad set_2 = \left[\begin{matrix}
  0 & 0 & 0 \\
  0 & 0.1 & 0 \\
   & \vdots &  \\
  0.1 & 0.1 & 0 \\
  & \vdots & \\
  0.9 & 0.9 & 0
 \end{matrix}\right]$$

In [4]:
def generate_points():
    # Generamos los valores de x1 y x2
    x1 = np.arange(start = 0, stop = 1,
                   step = 0.1, dtype=np.float64)
    x2 = np.arange(start = 0, stop = 1,
                   step = 0.1, dtype=np.float64)
    # Obtenemos una combinación de los puntos que hemos obtenido
    points2D = np.concatenate(np.array(np.meshgrid(x1,x2)).T)
    # Añadimos un cero por la izquierda y uno por la derecha respectivamente
    set1 = np.hstack((np.zeros(points2D.shape[0])[...,None], points2D))
    set2 = np.hstack((points2D, np.zeros(points2D.shape[0])[...,None]))
    # Y devolvemos un único conjunto de puntos
    return np.concatenate((set1, set2))

* **Apartado c:** para proyectar los puntos del mundo sobre el espacio 2D de la retina, tenemos que multiplicar los puntos del mundo por la matriz cámara que hemos generado, y normalizar estos puntos dividiendo por la última coordenada y devolviendo la coordenada $x_i$ e $y_i$ para el punto $X_i$ obteniendo así coordenadas 2D. Para ello, es necesario tener como entrada los puntos del mundo y la cámara, además de convertir estos puntos del mundo a coordenadas homogéneas. $$X' = PX = P \left[\begin{matrix}
x\\
y\\
z\\
1
\end{matrix}\right]$$ $$X'=\left[\begin{matrix}
x'\\
y'\\
z'
\end{matrix}\right] \qquad (u,v) = \left[\begin{matrix}\frac{x'}{z'}\\
\frac{y'}{z'}\end{matrix}\right]$$ Esta función devuelve las coordenadas homogéneas de los puntos del mundo y las coordenadas 2D de los puntos proyectados en la retina.

In [6]:
def project_points(points, camera):
    # Pasamos las coordenadas de los puntos a coordenadas homogéneas
    homogeneus_points = np.hstack((points, (np.ones(points.shape[0]))[...,None]))
    # Obtenemos una matriz vacía que serán las proyecciones
    # de los puntos al pasar por la cámara.
    projection = np.zeros(shape=(points.shape[0],2), dtype=np.float32)
    # Realizamos la multiplicación
    #    xy' = P * xy
    for i in range(homogeneus_points.shape[0]):
        point = np.dot(camera,homogeneus_points[i].T)
        projection[i,0] = point[0]/point[2]
        projection[i,1] = point[1]/point[2]

    # Devolvemos las proyecciones de los puntos
    return homogeneus_points, projection

* **Apartado d**: para realizar el algoritmo DLT (*Direct Linear Transformation*), partiremos de que conocemos los puntos reales del mundo (puntos xyz en coordenadas homogénas) y los puntos proyectados en la retina (en coordenadas 2D, que pasaremos a coordenadas homogéneas). Antes de comenzar con el algoritmo, debemos normalizar los puntos para que el resultado del DLT no se vea afectado por estos puntos. Esto se debe a que los puntos en coordenadas del mundo y coordenadas píxeles, pueden llegar a variar en varios órdenes de magnitud, haciendo que el espacio de búsqueda del algoritmo se vea incrementado. Si normalizamos estos puntos, el espacio de búsqueda queda muchísimo más acotado, permitiendo que el algoritmo obtenga mejores soluciones que si usamos los puntos sin tratar.

    Para normalizar estos puntos, se ha definido la función ```normalize```, que recibe como parámetros los puntos en coordenadas homogéneas, y para qué dimensión queremos normalizar, si 2D o 3D. Una vez aquí, calcularemos la media de los puntos y la desviación típica de estos, que usaremos para construir la matriz de normalización N: $$ N = \left(\begin{matrix}\sigma & 0 & 0 \end{matrix}\right)

In [4]:
# Los puntos han de ser en coordenadas homogéneas
def normalize(points, dim):
    # Obtenemos la media de los puntos y su desviación
    # típica para normalizar los datos
    points_mean = np.mean(points, 0)
    s = np.std(points[:,0:points.shape[1]-1])

    # Creamos la matriz N para normalizar los puntos, esta
    # matriz tiene la forma:
    if dim == 2:
        N = np.matrix([ [s, 0, points_mean[0]], [0, s, points_mean[1]], [0, 0, 1] ])
    else:
        N = np.matrix([[s, 0, 0, points_mean[0]], [0, s, 0, points_mean[1]], [0, 0, s, points_mean[2]], [0, 0, 0, 1]])

    N = np.linalg.inv(N)
    normalized_points = np.dot(N, points.T)
    normalized_points = normalized_points[0:dim,:].T
    
    return N, normalized_points

In [7]:

# Algoritmo DLT para obtener una cámara estimada a partir
# de los puntos en el mundo y los puntos de la retina
def DLT_algorithm(real_points, projected_points, camera):
    # Normalizamos los puntos para mejorar el resultado
    # del algoritmo DLT
    N_matrix, normalized_points = normalize(real_points, 3)
    homogeneus_proj_pt = np.hstack((projected_points, (np.ones(projected_points.shape[0]))[...,None]))
    N_matrix2d, norm_points_2d = normalize(homogeneus_proj_pt, 2)
    # Recorremos todos los puntos 3D que tenemos
    # y generamos una matriz M con todos los puntos
    aux = []
    for i in range(normalized_points.shape[0]):
        x_i, y_i, z_i = normalized_points[i,0], normalized_points[i,1], normalized_points[i,2]
        u, v = norm_points_2d[i,0], norm_points_2d[i,1]
        aux.append([x_i, y_i, z_i, 1, 0, 0, 0, 0, -u*x_i, -u*y_i, -u*z_i, -u])
        aux.append([0, 0, 0, 0, x_i, y_i, z_i, 1, -v*x_i, -v*y_i, -v*z_i, -v])

    # Descomponemos la matriz
    U, s, V = np.linalg.svd(np.array(aux, dtype=np.float64))
    # Obtenemos los parámetros
    camera_estimated = V[-1,:]/V[-1,-1]
    camera_estimated = np.matrix(camera_estimated).reshape(3,4)
    # Desnormalizamos
    camera_estimated = np.dot(np.dot(np.linalg.pinv(N_matrix2d), camera_estimated), N_matrix)
    camera_estimated = camera_estimated/camera_estimated[-1,-1]
    # Calculamos el error de la cámara estimada
    error = np.linalg.norm(x=(camera - camera_estimated), ord=None)
    
    return camera_estimated, error